## Machine Learning for Neuroscience, <br>Department of Brain Sciences, Faculty of Medicine, <br> Imperial College London
### Contributors: Francesca Palermo, Nan Fletcher-Lloyd, Alex Capstick, Yu Chen
**Winter 2022**



# Probability and Bayesian Theory

This tutorial will focus on probability and Bayesian theory using the machine learning library for Python scikit-learn (https://scikit-learn.org/stable).

## Import Dependencies

In [ ]:
import sklearn as sk
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Probability Theory

In this section, we will introduce you to hypothesis testing. 

Hypothesis testing is a statistical method that is used to make decisions using experimental data. 

A hypothesis is basically an assumption made about a population parameter and a test evaluates two mutually exclusive statements about a population to determine which statement is best supported by the data. 

These statements are the null and alternative hypothesis.

The null hypothesis suggests that no statistical significance exists in a set of given observations.

The alternative hypothesis is contrary to the null hypothesis. It is usually taken to be that the observations are the result of a real effect.

Read more about the key terms of hypothesis testing and the different types and when to use them here: https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce

Here, we will demonstrate how, with enough data, statistics can enable us to calculate probabilities using real-world observations. 

Here, probability provides the theory while statistics provides the tools to test that theory using data.

In the end, descriptive statistics such as the mean and standard deviation of the data become proxies for theoretical. This is because real-world probabilities are often quite difficult to calculate. As such, we rely on statistics and data.

With more and more data, we can become more confident that what we calculate represents the true probability of these events occurring.

So let's start with a question. For this, we are going to download the wine dataset.

In [ ]:
data = datasets.load_wine(as_frame=True)
features = data.data # this derives features as a dataframe (13 features by 178 instances)
features.columns = ['Alcohol','Malic Acid','Ash','Alcalinity of Ash','Magnesium','Total Phenols','Flavanoids', 'Nonflavanoid Phenols','Proanthocyanins','Color Intensity','Hue','OD280/OD315 of Diluted Wines','Proline']
labels = data.target # this derives labels as a dataframe (178 instances)

In [ ]:
wine = pd.concat([features, labels], axis=1,)
wine

Now, a quick check will remind us that there are three classes of wine in this dataset.

In [ ]:
labels.unique()

For this question, we will compare two of the three different types of wine across one feature variable, removing the need for scaling. 

So, let's pose our question. Is the level of alcohol different between classes 0 and 1?

First, we select for these classes and the column of interest.

In [ ]:
question = wine.iloc[:, [0,13]] # selects for the feature variable
question

Next, we need to group our 'question' dataframe by class. 

A quick check of the data shows that the target is an integer. To split out dataframe by group, we need this to be a str.

In [ ]:
question.dtypes

In [ ]:
question['target'] = question['target'].astype(str) #converts the target column to str

In [ ]:
question.dtypes

Now, we can groupby target and get each group out as a separate dataframe.

In [ ]:
grouped = question.groupby('target')
class_0 = grouped.get_group('0')
class_1 = grouped.get_group('1')

A quick check let's us know this has worked as planned.

In [ ]:
class_0

In [ ]:
class_1

We now want to use these values of the level of alcohol of the two different classes of wine to compare groups but these scores fall in a range. So how do we compare groups of values between types of wines and know with some degree of certainty that one is different from the other?

For this, we first need to understand the distribution of these data.

So let's visualise each group of values as histograms.

Learn more about using histograms here: https://seaborn.pydata.org/generated/seaborn.histplot.html

In [ ]:
df = pd.concat([class_0, class_1])
ax = sns.histplot(data=df, x="Alcohol", hue="target", element="step", bins=25, palette='PuRd')
sns.move_legend(ax, title='Class',loc='lower center', bbox_to_anchor=(0.5, -0.4), ncol=2)


Here we see that the data in each class is approximately normally distributed.

In probability, the normal distribution is a particular distribution of the probability across all of the data. The x-axis takes on the values of the data and the y-axis is the probability associated with each datapoint, from 0 to 1.

The high point of the distribution represents the datapoint with the highest probability of occurring. As you move further away from this point on either side, the probability of those points decreases, forming the familiar bell-shape curve. The high point in a statistical context actually represents the mean, and as you move farther from the mean, frequency rapidly decreases.

If any two distributions overlap significantly, we might assume they actually come from the same distribution and that there is no real difference in the means of those dsitributions. If there is no overlap, it is safe to assume that the distributions aren't the same. But, as you will see in the plot above, the difficulty is when there is some overlap.

The normal distribution is significant to probability and statistics due to two factors: the Central Limit Theorem and the Three Sigma Rule.

The key tenet of the Central Limit Theorem is that with more data, the closer the average of these data to the true probability. The Central Limit Theorem dictates that if we collect many data, the distribution of these data will look like a normal distribution and the high point of this distribution will align with the true value that the estimates should take on, i.e. the average of these many data will approach the true mean.

The Three Sigma Rule tells us how much the data will be spread out aroud this mean or how many of the data falls within a certain distance of the mean. The standard deviation is the average distance a singular measurement in the dataset is from the mean. The Three Sigma Rule dictates that given a normal distribution, 68% of the measurements will fall between one standard deviation of the mean, 95% within two, and 99.7% within three. This rule is also a statement of rarity of extreme values.

By taking advantage of the Three Sigma Rule, we can prescribe a value to how likely it is that the the level of alcohol is different within each class of wine.

Now, the next thing we want to do is turn our question into a null and alternative hypothesis, as this will allows us to carry out hypothesis testing.

Question: Is the level of alcohol different between classes 0 and 1?

So, our null hypothesis is that the level of alcohol is no different between classes 0 and 1 and our alternative hypothesis is that the level of alcohol is different between classes 0 and 1.

If you read through the article from earlier (https://towardsdatascience.com/hypothesis-testing-in-machine-learning-using-python-a0dc89e169ce) you will know that there are several different types of statistical tests that can be conducted to test a hypothesis.

Here, we want to test whether the means of two independent groups are equal or not. As such, we know we will be conducting a two sample, two-tailed test.

We are assuming that our data is normally distributed but we also know that the sample size of each group is greater than 30 so this does not matter quite so much. It is also possible to calculate the mean and standard deviation of both sample groups. Given this, we are going to run a z-test.

First, let's calculate the mean and standard deviation of the level of alcohol of each class of wine.

In [ ]:
from numpy import mean
from numpy import std

In [ ]:
class_0_avg = np.mean(class_0.Alcohol) # calculates variable mean
class_0_std = np.std(class_0.Alcohol) # calculates variable standard deviation
class_1_avg = np.mean(class_1.Alcohol)
class_1_std = np.std(class_1.Alcohol)

In [ ]:
print("Class 0: ", class_0_avg, class_0_std)
print("Class 1: ", class_1_avg, class_1_std)

These values look very similar! But let's calculate the z-test statistic before making any final decisions.

In [ ]:
from statsmodels.stats.weightstats import ztest

*N.B. you can learn more about using the statsmodels z-test here: https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ztest.html*

This method requires the dataframes to be in arrays.

In [ ]:
class_0 = class_0['Alcohol'].values
class_1 = class_1['Alcohol'].values

Finally, we can calculate the test-statistic and p-value.

In [ ]:
ztest(class_0, class_1, value=0, alternative='two-sided') 

The p-value tells us whether to accept or reject the null hypothesis. 100% accuracy is not possible for accepting or rejecting a hypothesis; therefore, we select a level of significance that is usually 5%. As such, if the p-value > 0.05, then the null hypothesis is accepted, but if the p-value < 0.05, then the null hypothesis is rejected.

In this case, we reject the null hypothesis. The level of alcohol is different between classes 0 and 1.

And that's it! You've now learnt the basics of hypothesis testing.

## Bayesian Theory

The second part of this tutorial will focus on Bayesian theory.

Now, as you will have learnt in your lectures, Naive Bayes methods are a set of supervised learning methods (classifiers) that assume that the presence of a particular feature in a class is unrelated to the presence of any other feature.

There are several types of Naive Bayes models (learn more here: https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) but for this exercise, we will focus on Gaussian Naive Bayes classifiers.

Learn more about how to use a Gaussian Naive Bayes classifier here: https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html#sklearn.naive_bayes.GaussianNB

For this, we will create our own dataset of 10 features across 500 instances, categorised into two classes. Class 0 represents a control group and Class 1 the class of interest. As in real world medical problems, there are fewer isntances of the class of interest.

Learn more about making your own classification datasets here: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html

In [ ]:
features, labels = datasets.make_classification(n_samples=500, n_features=10, n_informative=8, n_classes=2, weights=np.array([0.65, 0.35]), flip_y=0.01, class_sep=2.0, shuffle=True, random_state=42)

Now, let's take a quick look at the features and labels that were generated, in dataframe form for ease.

In [ ]:
features = pd.DataFrame(features)
labels = pd.DataFrame(labels)
df = pd.concat([features, labels], axis=1)
df.columns = ['F1','F2','F3','F4','F5','F6','F7','F8','F9','F10','Class']
df

Now, as we are working with Naive Bayes classifer, we do not need to do any feature scaling, so let's get straight into building our model. 

First, we must split our data into training and testing data.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state = 42)

Next, we need to import and build our model.

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB = GaussianNB(var_smoothing=0.5)

In [ ]:
GNB.fit(x_train, y_train) # fits the model on the training data
y_pred = GNB.predict(x_test) # predicts labels on the test data

Now, we want to evaluate the performance of the classification model.

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

In [ ]:
f1 = f1_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)

f1_avg = mean(f1_score(y_test, y_pred, average=None))
recall_avg = mean(recall_score(y_test, y_pred, average=None))
precision_avg = mean(precision_score(y_test, y_pred, average=None))

f1_sd = std(f1_score(y_test, y_pred, average=None))
recall_sd = std(recall_score(y_test, y_pred, average=None))
precision_sd = std(precision_score(y_test, y_pred, average=None))

print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1_avg)
print('recall_avg\t',recall_avg)
print('precision_avg\t',precision_avg)

print('\nf1_sd:\t\t',f1_sd)
print('recall_sd\t',recall_sd)
print('precision_sd\t',precision_sd)

print('\n',classification_report(y_test, y_pred))
print(roc_auc_score(y_test, y_pred))

Now, these scores are not very good, particularly when it comes to identifying the class of interest (Class 1). So how can we improve model performance?

Well, one method is to tune the hyperparameters of the model (read more about this here: https://medium.com/analytics-vidhya/how-to-improve-naive-bayes-9fa698e14cba)

For this, we first need to import the sklearn GridSearchCV function. This functions runs through all the different parameters fed into the parameter grid and produces the best combination of parameters based on a chosen scoring metric.

Learn more about how to use this function here: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

In [ ]:
from sklearn.model_selection import GridSearchCV

Next, we need to set the range of all the parameters we will feed into the parameter grid. For a Gaussian Naive Bayes classifier, this is just var_smoothing, which represents a stability calculation to widen (or smooth) the curve and therefore account for more samples that are further away from the distribution mean.

In [ ]:
param_grid_nb = {'var_smoothing': np.logspace(0,-9, num=1000)} # np.logspace returns numbers spaced evenly on a log scale, starts from 0, ends at -9, and generates 1000 samples

Now, we build the GridSearchCV, using the model and parameter grid. We then fit this searching tool to the training data using a 10-fold cross-validation to find an optimal combination of hyperparameters that minimizes a predefined loss function to give better results.

In [ ]:
GNB_Grid = GridSearchCV(estimator=GaussianNB(var_smoothing=0.5), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=-1)
GNB_Grid.fit(x_train, y_train)

In [ ]:
print(GNB_Grid.best_estimator_)

Now we want to re-evaluate the model on the test data.

In [ ]:
grid_pred = GNB_Grid.predict(x_test)

In [ ]:
f1 = f1_score(y_test, grid_pred)
recall = recall_score(y_test, grid_pred)
precision = precision_score(y_test, grid_pred)

f1_avg = mean(f1_score(y_test, grid_pred, average=None))
recall_avg = mean(recall_score(y_test, grid_pred, average=None))
precision_avg = mean(precision_score(y_test, grid_pred, average=None))

f1_sd = std(f1_score(y_test, grid_pred, average=None))
recall_sd = std(recall_score(y_test, grid_pred, average=None))
precision_sd = std(precision_score(y_test, grid_pred, average=None))

print('\nf1:\t\t',f1)
print('recall\t\t',recall)
print('precision\t',precision)

print('\nf1_avg:\t\t',f1_avg)
print('recall_avg\t',recall_avg)
print('precision_avg\t',precision_avg)

print('\nf1_sd:\t\t',f1_sd)
print('recall_sd\t',recall_sd)
print('precision_sd\t',precision_sd)

print('\n',classification_report(y_test, grid_pred))
print(roc_auc_score(y_test, grid_pred))

Now, these scores are much better!

And there you have it! You've now built and tuned your first Bayesian classifer.

Now you've finished this tutorial, follow the instructions and complete the assessment.